### Optimal Tactical Pathing for UGV

In [12]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import scipy.sparse as sp
import pandas as pd
import os
import csv
from PIL import Image

In [13]:
def two_step_optimization(w, h, s, f, A, N, d, t, E, τ, inflow, outflow, batteryCapacity):
    MAXTIME = 120
    
    # Step 1: Minimize detection to find the least-detection path
    m1 = gp.Model()
    m1.setParam('TimeLimit', MAXTIME)
    m1.setParam('MIPGap', 1e-2)
    m1.setParam('OutputFlag', 1)

    # Decision variables for detection path
    x = m1.addVars(A, vtype=GRB.BINARY, name="x")
    
    # Objective function: minimize detection cost
    m1.setObjective(gp.quicksum(d[i] * x[i] for i in range(A)), GRB.MINIMIZE)

    # Constraints
    for i in range(N):
        m1.addConstr(gp.quicksum(x[k] for k in inflow[i]) <= 1)
        m1.addConstr(gp.quicksum(x[k] for k in outflow[i]) <= 1)
        m1.addConstr(gp.quicksum(x[k] for k in inflow[i] if i != f and i != s) -
                      gp.quicksum(x[k] for k in outflow[i] if i != f and i != s) == 0)
    
    m1.addConstr(gp.quicksum(x[k] for k in inflow[s]) - gp.quicksum(x[k] for k in outflow[s]) == -1)
    m1.addConstr(gp.quicksum(x[k] for k in inflow[f]) - gp.quicksum(x[k] for k in outflow[f]) == 1)

    # tri_tot = len(tris)
    # for t in range(tri_tot):
    #     m1.addConstr(gp.quicksum(x[t] for t in tris[t]) <= 1)

    m1.addConstr(gp.quicksum(t[i] * x[i] for i in range(A)) <= τ)

    # Solve the optimization model
    m1.optimize()

    # Extract the least-detection path
    optimal_path = np.array([x[i].X for i in range(A)])
    path_cost = m1.objVal

    print("Least detection path:", optimal_path)
    print("Detection cost:", path_cost)

    # Step 2: Optimize generator usage along the given path
    m2 = gp.Model()
    m2.setParam('TimeLimit', MAXTIME)
    m2.setParam('MIPGap', 1e-2)
    m2.setParam('OutputFlag', 1)

    # Decision variables for battery levels and generator status
    batteryLevel = m2.addVars(A, lb=0, name="batteryLevel")
    y = m2.addVars(A, vtype=GRB.BINARY, name="y")
    
    # Objective function: minimize energy cost
    m2.setObjective(gp.quicksum(E[i] * optimal_path[i] * (1 - y[i]) for i in range(A)), GRB.MINIMIZE)

    # Constraints for battery levels
    m2.addConstr(batteryLevel[0] == batteryCapacity)
    for i in range(1, A):
        m2.addConstr(batteryLevel[i] == batteryLevel[i-1] - E[i-1] * optimal_path[i-1] * (1 - y[i-1]))
    
    for i in range(A):
        m2.addConstr(batteryLevel[i] >= 0)
        m2.addConstr(batteryLevel[i] <= batteryCapacity)

    m2.addConstr(gp.quicksum(E[i] * optimal_path[i] * (1 - y[i]) for i in range(A)) <= batteryCapacity)

    # Solve the optimization model
    m2.optimize()

    generator_decision = np.array([y[i].X for i in range(A)])
    final_objective = m2.objVal

    print("Generator decisions:", generator_decision)
    print("Final energy cost:", final_objective)

    return optimal_path, generator_decision, path_cost, final_objective


def write_path(opt_path, A, scenario_name):
    path = []
    for i in range(A):
        if opt_path[i] == 1:
            path.append(i)
    
    K = len(path)
    path_table = np.zeros((K, 2))
    for i in range(K):
        path_table[i] = [path[i], 1]  # Assuming you want to record path and some value
    
    path_df = pd.DataFrame(path_table, columns=["Node", "Value"])
    path_df.to_csv(f"output_{scenario_name}.csv", index=False)


In [14]:
SITUATION = "Buckner"
SPACING = 40

In [15]:
sat_path = os.path.join(os.getcwd(), 'imagery', SITUATION+'.png')
sat_map = np.asarray(Image.open(sat_path))
l, w, h = sat_map.shape
rows = l // SPACING
cols = w // SPACING


In [18]:
arcs = pd.read_csv('Buckner_arcs_3_3.csv')
arcs
ins_csv = pd.read_csv('Buckner_ins_3_3.csv', header=None)
outs_csv = pd.read_csv('Buckner_outs_3_3.csv', header=None)

ins_csv


,0,1,2,3,4,5,6,7,8,9
0,9,18,26,53,0,0,0,0,0,0
1,1,13,19,27,33,59,0,0,0,0
2,5,28,34,63,0,0,0,0,0,0
3,3,8,25,37,42,69,0,0,0,0
4,2,7,12,15,32,38,43,47,78,0
5,6,11,21,44,48,84,0,0,0,0
6,17,24,41,88,0,0,0,0,0,0
7,16,23,31,36,46,94,0,0,0,0
8,22,30,40,98,0,0,0,0,0,0
9,4,58,67,75,0,0,0,0,0,0


In [17]:
#User Input and Initial Calculations
inflow = []
outflow = []

# Loop through rows of the ins array
for i in range(ins_csv.shape[0]):
    new_in = []
    new_out = []
    
    # Loop through columns
    for j in range(10):
        if ins_csv[i, j] != 0:
            new_in.append(ins_csv[i, j])
            new_out.append(outs_csv[i, j])
    
    inflow.append(new_in)
    outflow.append(new_out)

t = 60*60*60
s = 1
f = 9
N=w*h*2
A=max(arcs['Arc'])
d = arcs['Risk']
E = arcs['Energy Level']
t = 60*60*60

batteryCapacity = 100000

KeyError: (0, 0)

In [ ]:
two_step_optimization(w, h, s, f, A, N, d, t, E, t, inflow, outflow, batteryCapacity)